<a href="https://colab.research.google.com/github/littleactuary/EAA/blob/main/hai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import plotly.express as px



data = pd.read_excel('a-hien.xlsx', header=2, index_col = 0)
data = data.set_axis(['x'], axis=1)
data['cdf'] = [0.5, 0.2, 0.1, 0.05, 0.025, 0.01, 0.005]
data.x *=-1
data.cdf = data.cdf*(-1)+1
print(data)
fig = px.line(data, x="x", y="cdf", title='Fonction de répartition')
fig.show()


                                         x    cdf
50% (1 fois tous les 2 ans)     -324917.38  0.500
20% (1 fois tous les 5 ans       613415.99  0.800
10% ( 1 fosi tous les 10 ans)    730273.14  0.900
5% (1 fois tous les 20ans)       932861.52  0.950
2,5% (1 fois tous les 40 ans)   1655584.41  0.975
1% (1 fois tous les 100 ans)    1825139.65  0.990
0,5% (1 fois tous les 200 ans)  2101988.20  0.995


In [26]:
import scipy
f = lambda x,mu,sigma: scipy.stats.norm(mu,sigma).cdf(x)
mu,sigma = scipy.optimize.curve_fit(f,data.x,data.cdf)[0]



/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning:

Covariance of the parameters could not be estimated




# **Inputs**

## import

In [ ]:
import numbers
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
pd.options.plotting.backend = "plotly"


In [ ]:
tx_conc = pd.read_excel('tx_conc.xlsx', header=0, index_col=0)['Taux conc sans cste sans plancher']
tx_chgt_encours = 0.0075
tc_brut = tx_conc + tx_chgt_encours
TS = forcage_TS.rename(columns={"forçage taux provisoire": "provisoire", "forçage taux définitif": "definitif"})
TS_brut = TS.definitif#.reindex(np.arange())

In [ ]:
historique = pd.read_excel('historique.xlsx', header=0, index_col=0)
historique = historique.rename(columns={
    "TAUX 3M": "taux court",
    "Tx Oblig,10A": "taux long",
    "Spread A": "spread A",
    "Ind,Actions" : "ind act cotee",
    "Tx dividendes" : "taux div act cotee",
    "Ind,Immo" : "ind imm",
    "Tx inflation" : "taux inflation"
    })



sc_eco = pd.read_excel('sc_eco.xlsx', header=0, index_col=0)
sc_eco = sc_eco.rename(columns={
    "Taux court": "taux court",
    "Taux long": "taux long",
    "AAA": "spread AAA",
    "AA": "spread AA",
    "A": "spread A",
    "Ind Cotees" : "ind act cotee",
    "Tx div Cotees" : "taux div act cotee",
    "Indice Placement 1" : "ind imm",
    "Inflation" : "taux inflation"
    })

base1 = historique[['taux court', 'taux long', 'taux inflation', 'spread A', 'taux div act cotee', 'ind act cotee', 'ind imm']]
base1['ind act cotee'] = base1['ind act cotee']/base1['ind act cotee'][-1]*100
base1['ind imm'] = base1['ind imm']/base1['ind imm'][-1]*100
base2 = sc_eco[['taux court', 'taux long', 'taux inflation', 'spread A', 'taux div act cotee', 'ind act cotee', 'ind imm']]
base_eco = pd.concat([base1, base2.iloc[1:, ]])
base_eco['perf act cotee hors div'] =perf(base_eco['ind act cotee'], 12, 1)
base_eco['perf imm yc loyer'] =perf(base_eco['ind imm'], 12, 1)

# base_eco dépend de l'historique et du sc éco. L'historique ne change pas, mais sc éco change en fonction du scénario. Il faut mettre à jour cette base à chaque scénario

In [ ]:
annee_SI = 2021
duree_proj_ans= 10
tmg = PM['TMG moyen/stock']
col_ts = ['tmg','tsp', 'tsp_floor', 'tsp_cap', 'tsd', 'tsd_floor', 'tsd_cap','tsd_net', 'tx_conc_net', 'ega', 'tx_rachat_conj']
taux_servis = pd.DataFrame(np.zeros(shape=(duree_proj_ans+2, len(col_ts))), columns=col_ts).set_index(np.arange(-1, 11)+annee_SI)
tsd_last = 0.02
tsd_prec = 0.02
# tx_conc_nets
tx_conc_last = 0.0151 - tx_chgt_encours
tx_conc_prec = 0.0176 - tx_chgt_encours
tmg_moyen = (PM['PM initiales']*tmg).sum() / PM['PM initiales'].sum()

# taux_servis : table des taux servis qui sera rempli au fur et a mesure
taux_servis.loc[annee_SI-1,] = [np.nan, np.nan, np.nan, np.nan, tsd_prec, np.nan, np.nan, tsd_prec - tx_chgt_encours, tx_conc_last, np.nan, np.nan]
taux_servis.loc[annee_SI,] =   [np.nan, np.nan, np.nan, np.nan, tsd_last, np.nan, np.nan, tsd_last - tx_chgt_encours, tx_conc_prec, np.nan, np.nan]

In [ ]:
# test rachat conjoncturel
for annee in np.arange(2022, 2031) :
  # forcage TS, à compléter avec les TS modélisés plus tard
  taux_servis.loc[annee, ['tsd', 'tsd_net', 'tx_conc_net']] = [TS.loc[annee, 'definitif'], TS.loc[annee, 'definitif']- tx_chgt_encours, tx_conc.loc[annee]]

  ega = EGA(taux_servis.loc[annee-1, 'tsd_net'], taux_servis.loc[annee-2, 'tsd_net'], taux_servis.loc[annee-1, 'tx_conc_net'], taux_servis.loc[annee-2, 'tx_conc_net'])
  rc = rachat_conj(alpha, beta, delta, rc_min, rc_max, ega)

  taux_servis.loc[annee, 'ega'] = ega
  taux_servis.loc[annee, 'tx_rachat_conj'] = rc

## functions

In [ ]:
# chercher par nom (name) dans une liste d'objets
# exemple : chercher la loi de rachat d'un model point parmi la liste de toutes les lois de rachats
def search(name, mylist):
  try :
    return [x for x in mylist if x['name'] == name][0]
  except:
    return 0

In [ ]:
# interpolation for converting annual data to monthly data
# input has to be a annual dataFrame (with one column ?)
# type: linear/polynomial/etc with specific order
def monthly_interp(input, type='linear', order=2) :
  last_row = len(input)
  res = input.set_index(np.arange(1, last_row+1, dtype=int)*12).reindex(np.arange(0, last_row*12+1, dtype=int))
  res.iloc[0,0]=0
  res = res.interpolate(type, order=order).iloc[1:, ]
  return res

In [ ]:
# ecart glissant absolu
# ts/tc : taux servi/taux conc de la dernière année, ts_ant/tc_ant : taux servi/taux conc de l'avant-dernière année
def EGA(ts, ts_ant, tc, tc_ant):
  return (ts+ts_ant-tc-tc_ant)/2

In [ ]:
def perf(x, basis_x, basis_perf):
  """
  x : series of variable
  basis_x: 1 if 'annual' , 12 if 'monthly'
  basis_perf : 1 if 'annual' , 12 if 'monthly'
  """
  return (x / x.shift(basis_x))**(1/basis_perf)-1

def find_eco(data_eco, ref):
  return data_eco[ref]

def eval_var_eco (var, data_eco) :
  res = find_eco(data_eco, var['ref'])
  window = int(var['window'])*12
  if var['op'] == 'decale':
    return res.shift(window, fill_value = 0) # may be changed ???
  else:
    # must be min, max, moyenne
    res = res.rolling(window)
    if var['op'] == 'moyenne':
      return res.mean()
    elif var['op'] == 'max':
      return res.max()
    else:
      return res.min()

def compute(formula, vars):
  for i in np.arange(0, len(vars)):
    formula = formula.replace('x{}'.format(i+1), 'vars[{}]'.format(i))
  return eval(formula)

# Epargne Euro

In [ ]:
PM_vec = PM['PM initiales']
PM_flat = PM_vec

# taux de versements libres potentiels (par année de souscription)
tx_vers_mensuel = (PM['Vers, libres']/PM_vec)/ 12

nb_subs_years = len(PM.index)
last_lapse_rate = rachats_structurels.iloc[-1][0]

zero_vec = pd.Series(np.zeros(shape=(nb_subs_years)), index = PM.index)

duree_proj_ans = 10
duree_proj_mois = duree_proj_ans*12
cols = ['PM flat', 'PM avec TMG', 'PM', 'primes',  'rachats', 'rachats_struc', 'rachats_conj', 'deces', 'echeances', 'chgt/encours', 'comm/encours', 'chgt/primes', 'comm/primes' , 'pen_rachat', 'CSG',
        'IC liq rachats', 'IC liq deces', 'IC liq echeances', 'PB liq rachat', 'PB liq deces', 'PB liq echeances']
# initier le tableau de projection
proj = pd.DataFrame(np.zeros(shape=(duree_proj_mois+1, len(cols))), columns=cols)
proj.iloc[0, 0] = PM_vec.sum()
proj.iloc[0, 1] = proj.iloc[0, 0]
proj.iloc[0, 2] = proj.iloc[0, 0]

col_ts = ['tsp', 'tsp_floor', 'tsp_cap', 'tsd', 'tsd_floor', 'tsd_cap','tsd_net', 'tmg']
taux_servi = pd.DataFrame(np.zeros(shape=(duree_proj_ans+1, len(col_ts))), columns=col_ts)

# tableau de PM par année de souscription, projeté par mois (ce sera utile ...)
for mois in np.arange(1,duree_proj_mois+1) :
  # m = 0, 1, 2, ..., 11
  m = (mois-1)%12
  CSG = 0
  # debut d'année
  if m == 0:
    # year n = 0, 1, 2, ...
    n = int((mois-1)/12)
    PM_flat = PM_vec
    PM_tmg = PM_vec
    # cumul des couts pour les assures (chargements et penalites) pendant l'année
    chgt_pen_cumul = zero_vec
    # TS provisoire
    tx_prov = TS.iloc[n].provisoire

    # EGA pour rachat conj
    # forcage TS, à compléter avec les TS modélisés plus tard
    annee = n + annee_SI +1
    taux_servis.loc[annee, ['tsd', 'tsd_net', 'tx_conc_net']] = [TS.loc[annee, 'definitif'], TS.loc[annee, 'definitif']- tx_chgt_encours, tx_conc.loc[annee]]

    ega = EGA(taux_servis.loc[annee-1, 'tsd_net'], taux_servis.loc[annee-2, 'tsd_net'], taux_servis.loc[annee-1, 'tx_conc_net'], taux_servis.loc[annee-2, 'tx_conc_net'])
    tx_rc_annuel = rachat_conj(alpha, beta, delta, rc_min, rc_max, ega)

    taux_servis.loc[annee, 'ega'] = ega
    taux_servis.loc[annee, 'tx_rachat_conj'] = tx_rc_annuel


  # chgt / encours
  chgt_encours_flat =  PM_flat * tx_chgt_encours/12
  chgt_encours_tmg =  PM_tmg * tx_chgt_encours/12
  chgt_encours_vec =  PM_vec * tx_chgt_encours/12

  # comm / encours
  comm_encours_vec = PM_vec * tx_comm_encours/12

  # calculs intermediaires
  revalo_tmg_demi_mois = (1+tmg)**(1/24)
  revalo_prov_demi_mois = (1+tx_prov)**(1/24)

  t_depuis_jan = (m+0.5)/12
  coef_ic_pb_liq = 1-(1+tx_prov)**(-t_depuis_jan)
  coef_ic_liq = 1-(1+tmg)**(-t_depuis_jan)

  # milieu de mois
  PM_flat = (PM_flat - chgt_encours_flat)
  PM_tmg = (PM_tmg - chgt_encours_tmg)*revalo_tmg_demi_mois
  PM_vec = (PM_vec - chgt_encours_vec)*revalo_prov_demi_mois



  # echeance
  #tx_echeance = echeances.shift(periods=-n, fill_value=last_echeance_rate).iloc[::-1].iloc[-nb_subs_years:].set_index(PM_vec.index).iloc[:,0] /12
  tx_echeance = echeances.iloc[n]/12

  echeance_vec = PM_vec*tx_echeance
  ic_pb_liq_echeance = echeance_vec*coef_ic_pb_liq
  ic_liq_echeance = echeance_vec*coef_ic_liq
  pb_liq_echeance = ic_pb_liq_echeance - ic_liq_echeance

  # deces
  tx_deces = table_morta.iloc[n]/12
  deces_vec = PM_vec*tx_deces
  ic_pb_liq_deces = deces_vec*coef_ic_pb_liq
  ic_liq_deces = deces_vec*coef_ic_liq
  pb_liq_deces = ic_pb_liq_deces - ic_liq_deces

  # rachat (structurel+ conj)
  # tx_rachat_conj = 1- (1-tx_rc_annuel)**(1/12)
  tx_rachat_conj = tx_rc_annuel/12
  # à modifier : taux rachat mensuel <> taux rachat annuel /12. il y a un lissage mensuel
  #tx_rachat_struc = rachats_structurels.shift(periods=-n, fill_value=last_lapse_rate).iloc[::-1].iloc[-nb_subs_years:].set_index(PM_vec.index).iloc[:,0]/12
  tx_rachat_struc = rachats_structurels.shift(periods=-mois, fill_value=last_lapse_rate).iloc[::-1].iloc[-nb_subs_years:].set_index(PM_vec.index).iloc[:,0]/12

  #tx_rachat = tx_rachat_struc + tx_rachat_conj
  #rachat_vec = PM_vec * tx_rachat
  rachat_struc_vec = PM_vec * tx_rachat_struc
  rachat_conj_vec = PM_vec * tx_rachat_conj
  rachat_vec = rachat_struc_vec + rachat_conj_vec
  pen_rachat = rachat_vec*tx_pen_rachat
  rachat_vec = rachat_vec - pen_rachat

  ic_pb_liq_rachat = rachat_vec*coef_ic_pb_liq
  ic_liq_rachat = rachat_vec*coef_ic_liq
  pb_liq_rachat = ic_pb_liq_rachat - ic_liq_rachat

  chgt_pen_cumul = chgt_pen_cumul + chgt_encours_vec + pen_rachat



  sorties_sans_pen_vec = echeance_vec + deces_vec + rachat_vec + pen_rachat
  PM_flat = PM_flat - sorties_sans_pen_vec
  PM_tmg = PM_tmg - sorties_sans_pen_vec
  PM_vec = PM_vec - sorties_sans_pen_vec

  # versements, affaires nouvelles

  versement =  PM_vec * tx_vers_mensuel
  chgt_primes = versement * tx_chgt_primes
  comm_primes = versement * tx_comm_primes
  versement = versement - chgt_primes


  # revalo fin de mois ou fin d'année
  PM_tmg = (PM_tmg+ versement) * revalo_tmg_demi_mois
  PM_vec = (PM_vec+ versement) * revalo_prov_demi_mois


  # si decembre
  if m == 11:
    tx_def = TS.iloc[n].definitif
    PM_vec = PM_vec*(1+tx_def)/(1+tx_prov)
    # prelevement CSG
    CSG_vec = (PM_vec - PM_flat- chgt_pen_cumul)*tx_CSG.iloc[n]
    PM_vec = PM_vec - CSG_vec
    CSG = CSG_vec.sum()

  proj.iloc[mois] = [PM_flat.sum(), PM_tmg.sum(), PM_vec.sum(), versement.sum(), rachat_vec.sum(), rachat_struc_vec.sum(), rachat_conj_vec.sum(), deces_vec.sum(), echeance_vec.sum(),
                     chgt_encours_vec.sum(), comm_encours_vec.sum(), chgt_primes.sum(), comm_primes.sum() , pen_rachat.sum(), CSG,
                     ic_liq_rachat.sum(), ic_liq_deces.sum(), ic_liq_echeance.sum(), pb_liq_rachat.sum(), pb_liq_deces.sum(), pb_liq_echeance.sum()]

In [ ]:
frais_generaux = frais_generaux.rename(columns={"Sur encours": "encours", "Sur aff, nouv,": "AN", "Sur versmt": "vers", "Sur décès": "deces", "Sur sorties": "sorties"})
tx_frais_generaux = frais_generaux.loc[np.repeat(frais_generaux.index, 12)].reset_index(drop=True).iloc[0:duree_proj_mois+1]


proj['frais/encours']=proj['PM'].mul(tx_frais_generaux['encours'], axis='index').shift(periods=1, fill_value=0)/12
proj['frais/versements']=proj['rachats'].mul(tx_frais_generaux['sorties'].shift(periods=1, fill_value=0), axis='index')
proj['frais/rachats']=proj['rachats'].mul(tx_frais_generaux['sorties'].shift(periods=1, fill_value=0), axis='index')
proj['frais/deces']=proj['deces'].mul(tx_frais_generaux['deces'].shift(periods=1, fill_value=0), axis='index')
proj['frais/echeances']=proj['echeances'].mul(tx_frais_generaux['sorties'].shift(periods=1, fill_value=0), axis='index')

In [ ]:
# alternative solution
frais_generaux = frais_generaux.rename(columns={"Sur encours": "encours", "Sur aff, nouv,": "AN", "Sur versmt": "vers", "Sur décès": "deces", "Sur sorties": "sorties"})
tx_frais_generaux = frais_generaux.loc[np.repeat(frais_generaux.index, 12)].reset_index(drop=True).iloc[0:duree_proj_mois+1].shift(periods=1, fill_value=0)
tx_frais_generaux['encours'] = tx_frais_generaux['encours']/12
assiette = proj[['PM', 'primes', 'primes', 'deces', 'rachats']]
assiette.columns = tx_frais_generaux.columns.values
frais = assiette*tx_frais_generaux


In [ ]:
# Cash assurance
frais = frais.sum(axis=1)
comm = proj['comm/encours']
presta = proj[['rachats', 'deces', 'echeances']].sum(axis=1)
primes = proj['primes']
pen = proj['pen_rachat']
cash = pd.DataFrame(primes + pen - presta - frais - comm, columns=['assurance'])

# cash divers :
# recettes diverses : variation des dettes diverses, nouveaux emprunts subordonnés, augmentations de capital
# ratio dettes diverses /PM fixe dans le temps --> remboursement mensuel de dettes diverses rentre (en négatif) dans recettes diverses
# dépenses diverses : variation des autres actifs et des matériels/mobiliers, rembousrement des emprunts subordonnés, remboursement de primpe de remboursement, intérêt versés (sur emprunts subordonnés, dividendes versés, IS)




In [ ]:
# IS
tx_IS = fiscalite['Taux IS CT'].loc[np.repeat(frais_generaux.index, 12)].reset_index(drop=True).iloc[0:duree_proj_mois+1].shift(periods=1, fill_value=0)
res_brut_IS = 0
res_net_IS = tx_IS*res_brut_IS




**Taux de référence.
Comment le définir de manière dynamique**



**et l'évaluer à chaque pas de temps ?**

Rate_ref : objet pour un taux de référence
name
els : liste des taux élémentaires utilisant dans la construction du taux de référence (par exemple : moyenne 5 ans des taux 10ans, taux court, ...)
els_values : pandas dataframe pour stocker les valeurs des taux élémentaires chaque année
formula : définition du taux à partir des taux éléments

In [ ]:
# taux de référence : name, vars [x1, x2, x3], formula
# eval(var_bdd) avec var_bdd tableau des valeurs x1, x2, x3 projetées dans le temps
class Taux_ref:
  def __init__(self, name, vars, formula):
    self.name = name
    self.vars = vars
    self.formula = formula


  def eval(self, var_bdd):
    xNames = list(map(lambda x: x['desc'] , self.vars))
    vars = var_bdd[xNames]
    duree = var_bdd.shape[0]
    res = np.zeros(duree)
    for i in np.arange(0, duree):
      res[i] = compute(self.formula, vars.iloc[i, :])
    vars.insert(0, "output", res, True)
    return vars


# coef A + taux de référence + coefB

class Rate_func:
    def __init__(self, a, ref, b):
        self.a = a
        self.b = b
        self.ref = ref
    def __str__(self):
      return f"{self.a :.2%} * {self.ref.name} {self.b:+.2%}"

    def print(self):
      return f"{self}"

    def eval(self, var_bdd):
      return self.a * self.ref.eval(var_bdd)['output'] + self.b



# objectif de taux servis
# target : une fonction de taux
# floor1 : taux plancher 1 -> un des taux de référence
# floor2 : taux plancher 2 -> propre au produit
# cap1 : taux plafond 1 -> un des taux de référence
# cap2 : taux plancher 2 -> propre au produit
class OTS:
    def __init__(self, target, floor1, floor2, cap1, cap2):
        self.target = target
        self.floor1 = floor1
        self.floor2 = floor2
        self.cap1 = cap1
        self.cap2 = cap2

    def print(self):
      return {
         'objectif': self.target.print(),
         'plancher 1': self.floor1.print(),
         'plancher 2': self.floor2.print(),
         'plafond 1': self.cap1.print(),
         'plafond 2': self.cap2.print()
      }

    def eval(self, var_bdd):
      target = self.target.eval(var_bdd)
      floor1 = self.floor1.eval(var_bdd)
      floor2 = self.floor2.eval(var_bdd)
      cap1 = self.cap1.eval(var_bdd)
      cap2 = self.cap2.eval(var_bdd)
      res = pd.DataFrame({'target': target, 'floor1': floor1, 'floor2': floor2, 'cap1': cap1, 'cap2': cap2})
      floor = res[['floor1', 'floor2']].max(axis=1)
      cap = res[['cap1', 'cap2']].min(axis=1)
      res.insert(1, 'floor', floor, True)
      res.insert(4, 'cap', cap, True)
      res=res.drop(['floor1', 'floor2', 'cap1', 'cap2'], axis=1)
      res['cap'] = res[['floor','cap']].max(axis=1)
      return res



In [ ]:
x1 = {'op': 'decale', 'window': 0 , 'ref': 'taux long', 'desc': 'decale_0_taux_long'}
x2 = {'op': 'decale', 'window': 1 , 'ref': 'taux court', 'desc': 'decale_1_taux_court' }
x3 = {'op': 'moyenne', 'window': 8, 'ref': 'taux long', 'desc': 'moyenne_8_taux_long' }
x4 = {'op': 'moyenne', 'window': 8, 'ref': 'spread A', 'desc': 'moyenne_8_spread_aa' }
x5 = {'op': 'moyenne', 'window': 2, 'ref': 'perf act cotee hors div', 'desc': 'moyenne_2_perf_act_net'}
x6 = {'op': 'decale', 'window': 0, 'ref': 'taux div act cotee', 'desc': 'decale_0_taux_div' }
x7 = {'op': 'decale', 'window': 0, 'ref': 'perf imm yc loyer', 'desc': 'decale_0_perf_imm' }

In [ ]:
# BDD variables pour fonction de taux perso
# il faut parcourir tous les taux ref définis et construire la BDD
nbYears = 40
xArray = [x1, x2, x3, x4, x5, x6, x7]
xNames = list(map(lambda x: x['desc'] , xArray))
proj = pd.DataFrame(index=base_eco.index, columns = xNames)

for x in xArray:
  proj[x['desc']] = eval_var_eco(x, base_eco)
# pas besoin d'une projection par mois. uniquement en décembre de chaque anee
var_bdd = proj.iloc[np.arange(11, nbYears*12+1, 12),:]
first_histo_year = 2001
# reset index to year only
var_bdd = var_bdd.set_index(np.arange(0, var_bdd.shape[0])+first_histo_year)
var_bdd


In [ ]:
var_bdd.plot().show()

**customized rate funtions**

In [ ]:
# test
TC = Taux_ref('taux_conc_net', [x1, x2, x3], "max(min(x3, x1), x2+0.01)")
res = TC.eval(var_bdd.loc[2021:2025,:])
objectif = Rate_func(1, TC, 0.05)
plancher1 = Rate_func(0, TC, 0.05)
plancher2 = Rate_func(0, TC, 0)
ots_1 = OTS(objectif, plancher1, plancher1, plancher1, plancher1)
res = ots_1.eval(var_bdd)

In [ ]:
def rachat_conj(alpha, beta, delta, rc_min, rc_max, ecart):
  """
  if (alpha > -0.04) or (alpha <-0.06) :
    raise Exception("alpha non compris entre -6% et -4%")
    return
  if(beta > 0) or (beta < -0.02) :
    raise Exception("beta non compris entre -2% et 0%")
    return
  if(delta > 0.04) or (delta < 0.02) :
    raise Exception("delta non compris entre 2% et 4%")
    return
  if(rc_min > -0.04) or (rc_min < -0.06) :
    raise Exception("Taux de RC min non compris entre -6% et -4%")
    return
  if(rc_max > 0.4) or (rc_max < 0.2) :
    raise Exception("Taux de RC max non compris entre 20% et 40%")
    return
  """

  gamma = 0.01
  if ecart < alpha:
    return rc_max
  elif ecart < beta:
    return rc_max*(ecart-beta)/(alpha-beta)
  elif ecart < gamma:
    return 0
  elif ecart < delta:
    return rc_min*(ecart-gamma)/(delta-gamma)
  else:
    return rc_min

In [ ]:
rachats_structurels = pd.read_excel('rachats_structurels.xlsx', header=0, index_col=0)
rachat_onc_moyen = {}

In [ ]:
# loi de rachat conjoncturel
alpha, beta, gamma, delta, rc_min, rc_max = -0.05, -0.005, 0.01, 0.03, 0, 0.423

In [ ]:

# liste des lois de rachats tructurels
liste_rachats_struc = [
    {'name': 'Multéo', 'data': rachats_structurels},
     {'name': 'Aviséo', 'data': rachats_structurels},
]

liste_rachats_conj = [
    {'name': 'ONC moyen', 'alpha':alpha, 'beta': beta, 'gamma': gamma, 'delta': delta, 'min': rc_min, 'max': rc_max},
    {'name': 'ONC min', 'alpha':alpha, 'beta': beta, 'gamma': gamma, 'delta': delta, 'min': rc_min, 'max': rc_max},
    {'name': 'ONC max', 'alpha':alpha, 'beta': beta, 'gamma': gamma, 'delta': delta, 'min': rc_min, 'max': rc_max},
]

liste_taux_ref = [

]



## <strong>Classe</strong>

Classe Epargne euro

In [ ]:
class Epargne_euro:
  def __init__(self, name, stock, TMGA, PB_contractuelle, charg_com, table_morta, echeance, loi_rachat_struc, loi_rachat_conj, taux_conc):
    self.name = name
    self.stock = stock # une ligne par année de souscription (on en déduit date début et fin de souscription). colonne 1 : PM, colonne 2 : versement potentiel, colonne 3 : TMG moyen
    self.tmga = TMGA
    self.csg_opt = csg_opt # CSG ? (true/false)
    self.pb_min_regl_option # inclus dans calcul PB min regl ? (true/false)
    self.pb_contractuelle = PB_contractuelle # {'pdt_fi': 0, 'res_tech_pos': 0, 'res_tech_neg':0}
    self.charg_com = charg_com # {'chgt_primes':0, 'comm_primes':0, 'chgt_encours':0, 'comm_encours':0, 'pen_rachat': 0 }
    self.table_morta = table_morta # année de simu en ligne, année de souscription en colonne
    self.echeance = echeance # comme rachat (ou décès, à voir...)
    self.loi_rachat_struc = search(loi_rachat_struc, liste_rachats_struc) # nom de la loi
    self.loi_rachat_conj = search(loi_rachat_conj, liste_rachats_conj) # nom de la loi. mapping à faire avec la liste des lois de rachats conjoncturels
    self.taux_conc = search(taux_conc, liste_taux_ref) # nom du taux de référence. Mapping à faire avec la liste des taux de référence

  def set_OTS(OTS):
    # OTS défini dans la politique de résultats
    self. = OTS.prov






##Tests

In [ ]:
table_morta = pd.read_excel('table_morta.xlsx', header=0, index_col=0)
stock = pd.read_excel('stock.xlsx', header=0, index_col=0)
rachats_structurels = pd.read_excel('rachats_structurels.xlsx', header=0, index_col=0)
echeances = pd.read_excel('echeances.xlsx', header=0, index_col=0)
forcage_TS = pd.read_excel('forcage_TS.xlsx', header=0, index_col=0)
frais_generaux = pd.read_excel('frais_generaux.xlsx', header=0, index_col=0)
fiscalite = pd.read_excel('fiscalite.xlsx', header=0, index_col=0)
rachats_structurels = pd.read_excel('rachats_structurels.xlsx', header=0, index_col=0)


tmg = PM['TMG moyen/stock']
TS = forcage_TS.rename(columns={"forçage taux provisoire": "provisoire", "forçage taux définitif": "definitif"})
tx_chgt_primes = 0.0028
tx_comm_primes = 0.001
tx_chgt_encours = 0.0075
tx_comm_encours = 0.0015
tx_pen_rachat = 0.005
tx_CSG = fiscalite['Taux CSG/CRDS']


In [ ]:
chgt_comm = {'chgt_primes': 0.0074 , 'comm_primes': 0, 'chgt_encours': 0.0048, 'comm_encours': 0, 'pen_rachats':0}

In [ ]:
name = 'Multéo'
cat = 'compte epargne'
csg_opt = True
pb_min_regl_opt = True
TMGA = 0.02
PB_contractuelle = {'pdt_fi': 0,  'res_tech_pos': 0, 'res_tech_neg':0}
chgt_comm = {'chgt_primes': 0.0074 , 'comm_primes': 0, 'chgt_encours': 0.0048, 'comm_encours': 0, 'pen_rachats':0}
stock = pd.read_excel('stock.xlsx', header=0, index_col=0) # stock.index (annees de souscription) stock.pm, stock.vers_potentiels, stock.tmg



TMG = 0.01

table_morta = 0
echeance = 0
loi_rachat_struc = 0
loi_rachat_conj = 0
taux_conc = 0
taux_ref_prov = 0.01
taux_provisoire = taux_prov
taux_cible = 'test taux cible'

Multeo = Epargne_euro('Multéo', stock, TMGA, PB_contractuelle, charg_com, TMG, table_morta, echeance, loi_rachat_struc, loi_rachat_conj, taux_conc, taux_provisoire, taux_cible )
res = Multeo.taux_provisoire.eval('31/01/2022')
res




In [ ]:
fig = px.line(rachats_structurels, title='taux de rachat par ancienneté')
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(z=table_morta.iloc[0:100], y=table_morta.index, x =table_morta.columns)])

fig.update_layout(title='table de mortalité', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
fig = px.line(table_morta.loc[2040], title='mortalité par année de souscription')
fig.show()

In [ ]:
fig = px.line(table_morta.loc[:, 1985], title='contrats souscrits en 1085 - mortalité par année de projection')
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(z=echeances.iloc[0:100], y=echeances.index, x =echeances.columns)])

fig.update_layout(title="taux d'échéance", autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()